In [351]:
import pandas as pd
import numpy as np
# import scipy.stat as sps
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import ExcelWriter

In [352]:
%%time
df = pd.read_excel('card transactions.xlsx')

Wall time: 11.2 s


In [353]:
# summary
numrecords = len(df)
stat_df = pd.DataFrame(df.isnull().sum(axis = 0))
stat_df = stat_df.reset_index().rename(columns = {'index':'variable', 0 :"# NaN"})
stat_df["% populated"] = (1 - stat_df["# NaN"]/numrecords)*100
stat_df["# unique"] = ''

for i in range(len(list(df))):
    stat_df["# unique"][i] = df[list(df)[i]].nunique()
    
stat_df

C:\Users\beizh\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,variable,# NaN,% populated,# unique
0,Recnum,0,100.000000,96753
1,Cardnum,0,100.000000,1645
2,Date,0,100.000000,365
3,Merchnum,3375,96.511736,13091
4,Merch description,0,100.000000,13126
5,Merch state,1195,98.764896,227
6,Merch zip,4656,95.187746,4567
7,Transtype,0,100.000000,4
8,Amount,0,100.000000,34909
9,Fraud,0,100.000000,2


In [354]:
df.head(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
5,6,5142149874,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.67,0
6,7,5142189277,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
7,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0
8,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0
9,10,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [355]:
##Identify any exclusions, bad records

# Remove single large transaction outlier

# Remove all but P transaction type
df = df[df['Transtype'] == 'P']

In [356]:
# --------------- Fill in NAs ----------

### Merchnum 
# Transform 0 values into NA
df.loc[df['Merchnum'] == '0', 'Merchnum'] = np.NaN

# fill most frequenct merchnum using merch description
dict1 = df.set_index('Merch description')['Merchnum'].to_dict()
df['Merchnum'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill most frequenct merchnum in this zip
dict1 = dict(zip(df['Merch zip'].dropna(), df['Merchnum']))
df['Merchnum'].fillna(df['Merch zip'].map(dict1), inplace = True)

pd.isnull(df['Merchnum']).sum()


1416

In [357]:
### Merch state and zip
numrecords - (df['Merch state'].isna() == df['Merch zip'].isna()).sum()


3809

In [359]:
### Merch zip
# df.groupby(['Merch description','Merch zip']).size()

# fill NA in Merch zip using Merch description
dict1 = df.set_index('Merch description')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch zip using Merchnum
dict1 = df.set_index('Merchnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch zip using Cardnum
dict1 = df.set_index('Cardnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Cardnum'].map(dict1), inplace = True)
# It will automatically choose zip with highest frequency

pd.isnull(df['Merch zip']).sum()


413

In [360]:
dic = dict(zip(df['Cardnum'], df['Merch zip']))
dic[5142310525]

60108.0

In [361]:
### Merch state
# df.groupby(['Merch description','Merch state']).size()
# Each Merch description has one matched state

# fill NA in Merch state using Merch description
dict1 = df.set_index('Merch description')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch state using Merchnum
dict1 = df.set_index('Merchnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch state using Merch zip

dict1 = dict(zip(df['Merch zip'].dropna(), df['Merch state']))
df['Merch state'].fillna(df['Merch zip'].map(dict1), inplace = True)

# fill NA in Merch state using Cardnum
dict1 = df.set_index('Cardnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Cardnum'].map(dict1), inplace = True)

pd.isnull(df['Merch state']).sum()


69

In [362]:
# Drop remained NA in three variables
df = df.dropna()

In [373]:
# df.groupby(['Cardnum','Date'])['Amount'].value_counts()
df[df['Cardnum'] == 5142190439]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
9,10,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
11,12,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
20,21,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.62,0
21,22,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
22,23,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.67,0
27,28,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
33,34,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
39,40,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [429]:
# --------------- Create variables ----------
### Amount expert variables
df2 = df.set_index('Date')
# Cardnum
# a = df2.groupby(['Cardnum']).rolling(window = '1D')['Amount'].mean().groupby(['Cardnum','Date']).mean().reset_index(['Cardnum','Date'])
# a = 
df2.groupby(['Cardnum','Date'])['Amount'].sum()
# .rolling(window = '0D').mean().reset_index(['Cardnum','Date'])

# df.join(a,on = ['Cardnum','Date'],rsuffix='_cardnum_1d_avg',how = 'left')
# a[a['Cardnum'] == 5142190439]


# data_range = lambda x: x.max() - x.min()
# ts.groupby(key).transform(data_range)


Cardnum     Date      
5142110081  2010-11-26     636.20
            2010-12-27     495.90
5142110402  2010-01-19     506.50
            2010-01-31      20.81
            2010-02-03      30.26
            2010-02-04     491.76
5142110434  2010-10-06    1551.64
5142110749  2010-04-19      62.94
            2010-06-13     794.85
5142110909  2010-01-17     351.24
            2010-01-20      64.40
            2010-02-09     301.79
            2010-03-21     128.00
            2010-03-22     857.87
            2010-04-05     435.47
            2010-04-12     191.65
            2010-04-13     809.45
            2010-05-18     159.60
            2010-06-12     288.00
            2010-07-07    1535.40
            2010-07-12      51.75
            2010-12-20     107.76
            2010-12-27     692.25
            2010-12-30     354.94
5142111097  2010-02-02      56.98
            2010-03-02      83.80
            2010-03-13      64.00
            2010-04-19      17.43
            2010-06-27   

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(df)? (<ipython-input-343-212e9880de12>, line 4)